
# EuroPython 2016 session instructions to PDF

# Session Chair Information

## Before your session

1. Plan to be in the talk room at least **10 minutes** before your session.
2. **Double check** that you have the **schedule** with you.
3. Walk to your room.
4. **Check** if there is spare water bottles for the speakers and video adaptors. If something is missing kindly ask the room manager to pick it.
5. Reach for the next speaker.

## For Each Talk

1. Ask the speaker how they would like to be introduced, and double check that you know how to pronounce their name.
2. Ask the speaker about their Q&A preference (see options below) and how they want to be announced of the last 10 or 5 minutes of talk.
3. **Make sure the talk starts on time!** At the talk's scheduled start time, get up on stage, get the microphone, make eye contact with the A/V crew to get their go-ahead, the introduce the speaker.
4. **Watch the clock.** Announce the speaker that their talk is about to finish, they way you agreed.
5. **Two minutes before the talk's scheduled end time**, if it doesn't seem like the speaker is going to stop on their own, quietly get up and stand to the side of the stage. Be ready to interrupt them if they do not finish on time: "I'm very sorry but our time is almost up.", "Can I suggest continuing in an open space?", or "Let's move on to Q&A.".
6. If the speaker requested **Q&A, stand beside the microphone to help moderate** (see section below).
7. When the speaker is finished: start the applause, then stand up and thank them.

## After Your Session

1. Go tell the coordinator or anyone in the reception desk that you finished your session.


## Q&A

Speakers might have different preferences on how they want to Q&A session to be. Please ask them. Here go 3 examples:
1. No questions; they will speak for the duration of their talk then leave.
2. Take a session-chair-moderated questions for the last 5 minutes of the timeslot.


## If You Have Any Problems

If you have any problems, concerns, or questions, try to contact someone on the `europython-volunteers` Telegram channel (http://bit.ly/28QF0FM).

Otherwise you can contact:
1. Christian Barra: barrachris@gmail.com / @christianbarra
2. Alexandre Savio: +34630555357 / alexsavio@gmail.com / @alexsavio
3. TODO Add someone else with a phone number
4. Any conference staff member.


## Moderating Q&A

To moderate a Q&A section, please:

1. Step up to the audience mic and say (something to the effect of): "Thank you {speaker}! We've got a few minutes for questions now".
2. Kindly ask the speaker to repeat the question once they heard it.
3. Questions should be kept short. If the discussion is taking too long (2 minutes), kindly ask to discuss after the talk and reach for the following question, if any.

In [16]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [17]:
import json
import datetime as dt
from   operator import itemgetter
from collections import OrderedDict
from operator import itemgetter

from IPython.display import display, HTML
from nbconvert.filters.markdown import markdown2html

from eptools._utils import DefaultOrderedDict

In [18]:
talk_sessions = json.load(open('accepted_talks.json'), encoding='utf-8')
talks_admin_url = 'https://ep2016.europython.eu/admin/conference/talk'

In [19]:
show = lambda s: display(HTML(s))

def ordinal(n):
    if 10 <= n % 100 < 20:
        return str(n) + 'th'
    else:
        return  str(n) + {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(n % 10, "th")


def talk_schedule(start, end):
    input_format  = "%Y-%m-%d %H:%M:%S"
    output_format_day = "%A, %B"
    output_format_time = "%H:%M"
    
    output_date = lambda d: "{} {} at {}".format(d.strftime(output_format_day), 
                                                 ordinal(int(d.strftime('%d'))),
                                                 d.strftime(output_format_time))
    start_date = dt.datetime.strptime(start, input_format)
    end_date   = dt.datetime.strptime(end  , input_format)

    return output_date(start_date), output_date(end_date)


def show_talk(talk, show_duration=True, show_link_to_admin=True):
    
    speakers  = talk['speakers']
    title     = talk['title']
    abstract  = talk['abstract_long'][0]
    room      = talk.get('track_title', '').split(', ')[0]
    timerange = talk.get('timerange', '').split(';')[0]
    
    pdf = ''
    pdf += '<h2>- {}</h2>\n'.format(title)
    
    if show_link_to_admin:
        pdf += talks_admin_url + '/{}'.format(talk['id'])
        pdf += '<a href={0}>{0}</a>\n'.format(talk_admin_url)
    
    if show_duration:
        pdf += '{} mins.'.format(talk['duration'])
    else:
        pdf += ''

    timerange = talk['timerange'].split(';')[0]
    try:
        start, end = talk_schedule(*timerange.split(', '))
    except:
        start, end = ('', '')

    if start:
        pdf += '<p>'
        pdf += '{} in {}'.format(start, room)
        if show_duration:
            pdf += ' ({})'.format(duration)
        pdf += '</p>\n'

        #show(schedule)
    
    pdf += '<h3><i>{}</i></h2>\n'.format(speakers)
    pdf += '<p>{}</p>\n'.format(markdown2html(abstract))
    pdf += '<br/>\n'
    return pdf

## Filter talks

In [20]:
sessions_talks = OrderedDict()

# remove the IDs from the talks
for name, sess in talk_sessions.items():
    sessions_talks[name] = [talk for tid, talk in sess.items()]

talks = sessions_talks['talk']

## Create start time and session fields in each talk

In [21]:
# add 'start' time for each talk
for idx, talk in enumerate(talks):
    tr = talk['timerange']
    if not tr:
        talk['start'] = dt.datetime.now()
    else:
        talk['start'] = dt.datetime.strptime(tr.split(',')[0].strip(), "%Y-%m-%d %H:%M:%S")
    talks[idx] = talk

    
# add 'session_code' for each talk
conference_start   = dt.date(2016, 7, 17)
first_coffee_start = dt.time(10,  0)
lunch_start        = dt.time(12, 45)
secnd_coffee_start = dt.time(15, 30)
close_start        = dt.time(18,  0)

journee_start_times = [first_coffee_start,
                       lunch_start,
                       secnd_coffee_start,
                       close_start]


def get_journee_number(talk_start_time, journee_start_times):
    for idx, start in enumerate(journee_start_times):
        if talk_start_time < start:
            return idx
    return -1
        
tracks = ['A1',
          'A2',
          'Barria 1',
          'Barria 2',
          'PyCharm Room',
          ]

for idx, talk in enumerate(talks):
    talk_start = talk['start'].time()
    talk_room  = talk['track_title'].split('[')[0].strip().replace(' ', '_')
    
    day_num     = (talk['start'].date() - conference_start).days
    journee_num = get_journee_number(talk['start'].time(), journee_start_times)
    talk['session'] = str(talk_room) + '_' + str(int(day_num)) + '.' + str(journee_num)
    talks[idx] = talk


## Group talks by session code

In [22]:
# sort by and group by session
keys = ('start', 'session')
sorted_talks = sorted(talks, key=itemgetter(*keys))

talk_sessions = DefaultOrderedDict(list)
for talk in sorted_talks:
    talk_sessions[talk['session']].append(talk)

## Create the HTML texts for each session

In [23]:
session_texts = OrderedDict()

for session, talks in talk_sessions.items():
    text = ['<h1>' + session + '</h1>']
   
    for talk in talks:      
        text += [show_talk(talk, show_duration=False, show_link_to_admin=False)]
        
    session_texts[session] = '\n'.join(text)

## Export to PDF

You need to have `pandoc`, `wkhtmltopdf` and `xelatex` installed in your computer.

In [25]:
import os
import os.path as op
import subprocess

os.makedirs('session_pdfs', exist_ok=True)

def pandoc_html_to_pdf(html_file, out_file, options):  
    cmd = 'pandoc {} {} -o {}'.format(options, html_file, out_file)
    print(cmd)
    subprocess.check_call(cmd, shell=True)


# pandoc options DIN-A6
# options = ' -V '.join(['-V geometry:paperwidth=6cm',
#                        'geometry:paperheight=8cm',
#                        'geometry:width=5.5cm',
#                        'geometry:height=7.5cm',
#                        'geometry:left=.25cm',
#                        ])

# pandoc options DIN-A4
options = ' -V '.join(['-V geometry:paperwidth=210mm',
                       'geometry:paperheight=297mm',
                       'geometry:left=2cm',
                       'geometry:top=2cm',
                       'geometry:bottom=2cm',
                       'geometry:right=2cm',
                       ])
options += ' --latex-engine=xelatex'

for session, text in session_texts.items():  
    html_file = op.join('session_pdfs', '{}.html'.format(session))
    out_file  = html_file.replace('.html', '.pdf')

    ops = open(html_file, mode='w')
    ops.write(text)
    ops.close()
    
    pandoc_html_to_pdf(html_file, out_file, options)
    
    os.remove(html_file)

pandoc -V geometry:paperwidth=210mm -V geometry:paperheight=297mm -V geometry:left=2cm -V geometry:top=2cm -V geometry:bottom=2cm -V geometry:right=2cm --latex-engine=xelatex session_pdfs/A1_1.1.html -o session_pdfs/A1_1.1.pdf
pandoc -V geometry:paperwidth=210mm -V geometry:paperheight=297mm -V geometry:left=2cm -V geometry:top=2cm -V geometry:bottom=2cm -V geometry:right=2cm --latex-engine=xelatex session_pdfs/A2_1.1.html -o session_pdfs/A2_1.1.pdf
pandoc -V geometry:paperwidth=210mm -V geometry:paperheight=297mm -V geometry:left=2cm -V geometry:top=2cm -V geometry:bottom=2cm -V geometry:right=2cm --latex-engine=xelatex session_pdfs/Barria_1_1.1.html -o session_pdfs/Barria_1_1.1.pdf
pandoc -V geometry:paperwidth=210mm -V geometry:paperheight=297mm -V geometry:left=2cm -V geometry:top=2cm -V geometry:bottom=2cm -V geometry:right=2cm --latex-engine=xelatex session_pdfs/Barria_2_1.1.html -o session_pdfs/Barria_2_1.1.pdf
pandoc -V geometry:paperwidth=210mm -V geometry:paperheight=297mm -V